In [1]:
import os

os.environ["NEO4J_URI"] = "neo4j+s://7297c836.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "6X6ZtF27S5Mt6-nuWQ4MjDGXhQI7UJaglEAfA_VbBQI"
os.environ["OPENAI_API_KEY"] = "yourkey"

In [8]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 6.6 MB/s eta 0:00:00


In [9]:
from langchain_community.document_loaders import PyPDFLoader
import glob

docs = []

for file in glob.glob("/content/*.pdf"):
    loader = PyPDFLoader(file)
    docs.extend(loader.load())

len(docs)

26

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = splitter.split_documents(docs)
len(chunks)

116

In [11]:
!pip install huggingface_hub --quiet
from huggingface_hub import login

login()

In [12]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings

model = SentenceTransformer("google/embeddinggemma-300m")

class GemmaEmbeddings(Embeddings):
    def embed_documents(self, texts):
        return model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        return model.encode([text], convert_to_numpy=True).tolist()

embeddings = GemmaEmbeddings()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

3_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

In [13]:
import numpy as np

class GemmaEmbeddingsNormalized(GemmaEmbeddings):
    def embed_documents(self, texts):
        vectors = super().embed_documents(texts)
        vectors = [v/np.linalg.norm(v) if np.linalg.norm(v) > 0 else np.zeros_like(v) for v in vectors]
        return vectors

    def embed_query(self, text):
        vector = super().embed_query(text)[0]
        return vector/np.linalg.norm(vector) if np.linalg.norm(vector) > 0 else np.zeros_like(vector)
embeddings = GemmaEmbeddingsNormalized()


In [14]:
for idx, chunk in enumerate(chunks):
    chunk.metadata['chunk_id'] = idx

In [16]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.3/325.3 kB 6.6 MB/s eta 0:00:00


In [17]:
from langchain_community.vectorstores import Neo4jVector

vectorstore = Neo4jVector.from_documents(
    documents=chunks,
    embedding=embeddings,
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    index_name="essay_chunkAgentSpace2",
    node_label="Chunk",
    text_node_property="text",
    embedding_node_property="embedding"
)


In [18]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"]
)

chunks_by_source = {}
for chunk in chunks:
    source = chunk.metadata.get('source', 'unknown')
    if source not in chunks_by_source:
        chunks_by_source[source] = []
    chunks_by_source[source].append(chunk)

print("Creating NEXT relationships...")
relationship_count = 0

for source, source_chunks in chunks_by_source.items():
    source_chunks.sort(key=lambda x: x.metadata.get('chunk_id', 0))

    for i in range(len(source_chunks) - 1):
        current_id = source_chunks[i].metadata['chunk_id']
        next_id = source_chunks[i + 1].metadata['chunk_id']

        query = """
        MATCH (c1:Chunk)
        WHERE c1.chunk_id = $current_id
        MATCH (c2:Chunk)
        WHERE c2.chunk_id = $next_id
        MERGE (c1)-[:NEXT]->(c2)
        """

        graph.query(query, params={
            "current_id": current_id,
            "next_id": next_id
        })
        relationship_count += 1

print(f"Created {relationship_count} NEXT relationships")

print("Creating Document nodes...")

for source, source_chunks in chunks_by_source.items():
    doc_query = """
    MERGE (d:Document {name: $source})
    SET d.chunk_count = $chunk_count
    """

    graph.query(doc_query, params={
        "source": source,
        "chunk_count": len(source_chunks)
    })

    for chunk in source_chunks:
        link_query = """
        MATCH (d:Document {name: $source})
        MATCH (c:Chunk)
        WHERE c.chunk_id = $chunk_id
        MERGE (c)-[:PART_OF]->(d)
        """
        graph.query(link_query, params={
            "source": source,
            "chunk_id": chunk.metadata['chunk_id']
        })

print(f"Created {len(chunks_by_source)} Document nodes")

def get_chunk_context(chunk_id, context_size=1):
    """Get neighboring chunks using NEXT relationships"""
    query = """
    MATCH (c:Chunk)
    WHERE c.chunk_id = $chunk_id
    OPTIONAL MATCH path_before = (before:Chunk)-[:NEXT*1..%d]->(c)
    OPTIONAL MATCH path_after = (c)-[:NEXT*1..%d]->(after:Chunk)
    WITH c,
         collect(DISTINCT before) as before_chunks,
         collect(DISTINCT after) as after_chunks
    RETURN
        c.text as current_text,
        c.chunk_id as current_id,
        [b in before_chunks | b.text] as before_texts,
        [a in after_chunks | a.text] as after_texts
    """ % (context_size, context_size)

    result = graph.query(query, params={"chunk_id": chunk_id})
    return result[0] if result else None

def hybrid_search(query_text, k=4, context_size=1):
    """Combine vector search with graph context"""
    results = vectorstore.similarity_search_with_score(query_text, k=k)

    hybrid_results = []
    for doc, score in results:
        chunk_id = doc.metadata.get('chunk_id')

        context = get_chunk_context(chunk_id, context_size) if chunk_id is not None else None

        hybrid_results.append({
            'text': doc.page_content,
            'score': score,
            'chunk_id': chunk_id,
            'source': doc.metadata.get('source', 'unknown'),
            'context': context
        })

    return hybrid_results

/tmp/ipython-input-2681258656.py:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-neo4j package and should be used instead. To use it run `pip install -U `langchain-neo4j` and import as `from `langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


Creating NEXT relationships...
Created 111 NEXT relationships
Creating Document nodes...
Created 5 Document nodes


In [19]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [23]:
query = "What specific argument does the author make about orchids"
results = retriever.invoke(query)

for i, r in enumerate(results):
    print(f"---- Result {i+1} ----")
    print(r.page_content[:300])

print("\n\n=== HYBRID SEARCH WITH CONTEXT ===")
hybrid_results = hybrid_search(query, k=4, context_size=1)

for i, result in enumerate(hybrid_results, 1):
    print(f"\n---- Hybrid Result {i} ----")
    print(f"Score: {result['score']:.4f}")
    print(f"Source: {result['source']}")
    print(f"Text: {result['text'][:200]}...")

    if result['context']:
        ctx = result['context']
        print(f"Context: {len(ctx.get('before_texts', []))} before, {len(ctx.get('after_texts', []))} after chunks")

---- Result 1 ----
Cai 4 them. Resist, he says, the temptations of the ornate, the intricate, the pretty. Ultimately, the best thing that people ought to do is to be cautious of things that are too good to be true, to see the underlying flesh and deceit that the pleasant scent and glamor conceal. Only then can one sur
---- Result 2 ----
paradigm of orchids being pleasant and inviting. By describing the orchids as disgusting and putrid things, Marlowe separates his opinions from the conventional wisdom of the plant. This dense, humid forest of orchids serves as an allegory for Los Angeles, with its terrible smell “as overpowering as
---- Result 3 ----
Cai 2 thick, wet, steamy and larded with the cloying smell of tropical orchids in bloom […] The plants filled the space, a forest of them, with nasty meaty leaves and stalks like the newly washed fingers of dead men. They smelled as overpowering as boiling alcohol under a blanket” (Chandler 7). Marl
---- Result 4 ----
and crude world aroun